In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
import wandb
from wandb import Api
from tqdm import tqdm  # Optional: for progress bar

def load_api_key():
    """Load the WANDB API key from the .env file."""
    load_dotenv()
    api_key = os.getenv('WANDB_API_KEY')
    if not api_key:
        raise ValueError("WANDB_API_KEY not found in the .env file.")
    return api_key

def initialize_api(api_key):
    """Initialize the WandB API client."""
    api = Api(api_key=api_key)
    return api

def main():
    # Load API key
    api_key = load_api_key()

    # Initialize API
    api = initialize_api(api_key)

    # Specify your entity and project
    entity = 'llma-agents-2024-the-agents-company'
    project = 'llma-agents-debug'

    # Fetch runs
    runs = api.runs(path=f"{entity}/{project}")

    if not runs:
        print("No runs found for the specified entity/project.")
        return None

    return runs


In [ ]:
run_collection = main()
run_collection_list: wandb.apis.public.runs.Run = [run for run in run_collection]

In [ ]:
run_collection_list

In [ ]:
import json


def runs_to_dataframe_full(runs):
    """
    Convert a list of wandb Run objects to a pandas DataFrame with all attributes.

    Args:
        runs (list): List of wandb Run objects.

    Returns:
        pd.DataFrame: DataFrame containing all run attributes.
    """
    data = []
    for run in runs:
        # Ensure attributes are loaded
        run_attrs = run._attrs.copy()  # Copy to avoid modifying the original run object

        # Flatten the 'user' nested dictionary
        user = run_attrs.get('user')
        if isinstance(user, dict):
            run_attrs['user_name'] = user.get('name')
            run_attrs['user_username'] = user.get('username')
            del run_attrs['user']  # Remove the nested 'user' dict

        # Parse JSON strings in 'summaryMetrics' and 'systemMetrics'
        for key in ['summaryMetrics', 'systemMetrics']:
            if key in run_attrs and isinstance(run_attrs[key], str):
                try:
                    run_attrs[key] = json.loads(run_attrs[key])
                except json.JSONDecodeError:
                    run_attrs[key] = None  # Handle invalid JSON

        # If there are other nested dictionaries, they can be handled here similarly

        data.append(run_attrs)

    # Use pandas.json_normalize to flatten any remaining nested structures
    df = pd.json_normalize(data)

    # Optional: Rename columns for better readability
    # For example, flatten 'historyKeys.lastStep' to 'historyKeys_lastStep'
    df.columns = [col.replace('.', '_') for col in df.columns]

    return df

In [ ]:
from typing import MutableMapping


def flatten_dict(d, parent_key='', sep='.'):
    """
    Recursively flattens a nested dictionary or list.

    Args:
        d (dict or list): The dictionary or list to flatten.
        parent_key (str): The base key string.
        sep (str): Separator between keys.

    Returns:
        dict: A flattened dictionary with dot-separated keys.
    """
    items = []
    if isinstance(d, MutableMapping):
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            items.extend(flatten_dict(v, new_key, sep=sep).items())
    elif isinstance(d, list):
        for idx, item in enumerate(d):
            new_key = f"{parent_key}{sep}{idx}" if parent_key else str(idx)
            items.extend(flatten_dict(item, new_key, sep=sep).items())
    else:
        items.append((parent_key, d))
    return dict(items)

In [ ]:
def runs_to_dataframe(runs):
    """
    Convert a list of wandb Run objects to a pandas DataFrame with specified fields.

    Args:
        runs (list): List of wandb Run objects.

    Returns:
        pandas.DataFrame: DataFrame containing run information and metrics.
    """
    run_data = []

    for run in runs:
        run_dict = {}

        # Extract required fields with default values if not present
        run_dict['id'] = getattr(run, 'id', None)
        run_dict['tags'] = ','.join(run.tags) if getattr(run, 'tags', None) else ''
        run_dict['name'] = getattr(run, 'name', None)
        run_dict['displayName'] = getattr(run, 'displayName', None)
        run_dict['sweepName'] = getattr(run, 'sweepName', None)
        run_dict['commit'] = getattr(run, 'commit', None)
        run_dict['createdAt'] = getattr(run, 'createdAt', None)
        run_dict['state'] = getattr(run, 'state', None)

        # Extract and flatten config metrics
        config = getattr(run, 'config', {}) or {}
        flattened_config = flatten_dict(config)
        for key, value in flattened_config.items():
            run_dict[f'config.{key}'] = value

        # Extract and flatten summaryMetrics
        summary_metrics = getattr(run, 'summaryMetrics', {}) or {}
        flattened_summary = flatten_dict(summary_metrics)
        for key, value in flattened_summary.items():
            run_dict[f'summaryMetrics.{key}'] = value

        run_data.append(run_dict)

    # Create DataFrame from the list of dictionaries
    df = pd.DataFrame(run_data)

    return df


In [92]:
type(run_collection_list[0])

wandb.apis.public.runs.Run

In [93]:
df = runs_to_dataframe(run_collection_list)

In [94]:
run_collection_list[23].config

{'Dspy': {'telepropmter': {'type': 'MIPROv2'}},
 'debug': True,
 'weave.run.name': 'fea6a66f',
 'Agent A Persona': 'You are a diligent, reliable, and knowledgeable.',
 'Agent B Persona': 'You are a attentive and detail-oriented.',
 'Agent C Persona': 'You have unique problem-solving abilities and can think out of the box.',
 'Agent D Persona': 'You are a strategic synthesizer who integrates multiple perspectives, systematically connecting complex ideas and uncovering deeper insights while maintaining intellectual flexibility and rigorous analytical precision.',
 'Agent E Persona': 'You are a transformative catalyst who bridges conceptual gaps, challenges conventional thinking, and generates innovative solutions by reframing problems through multiple lenses of creativity and critical analysis.',
 'ExchangeOfThought': {'mode': 'Report', 'rounds': 1}}

In [95]:
print(len(df.columns))
pd.Series(df.columns)

44


0                                                    id
1                                                  tags
2                                                  name
3                                           displayName
4                                             sweepName
5                                                commit
6                                             createdAt
7                                                 state
8                         config.Dspy.telepropmter.type
9                         config.ExchangeOfThought.mode
10                      config.ExchangeOfThought.rounds
11                                         config.debug
12                               config.Agent A Persona
13                               config.Agent B Persona
14                               config.Agent C Persona
15                               config.Agent D Persona
16                               config.Agent E Persona
17                                config.weave.r

In [196]:
# Now this df contains all relevant information about the reuns
df_copy = df.copy(deep=True)
#df_copy

# Specifiy Code for runs by Amaan

In [197]:
df_copy = df_copy[df_copy['state'] == 'finished']

In [198]:
df_copy = df_copy[df_copy['config.api'] == 'openai']

In [199]:
df_copy = df_copy[df_copy['sweepName'] == '63vzqpmb']

In [200]:
df_copy = df_copy[~((df_copy['config.ExchangeOfThought.mode'] == 'single') &
                   (df_copy['config.ExchangeOfThought.rounds'] > 1))]

In [201]:
df_copy.shape

(18, 44)

In [202]:
df_copy

,id,tags,name,displayName,sweepName,commit,createdAt,state,config.Dspy.telepropmter.type,config.ExchangeOfThought.mode,...,config.C,config.D,config.E,config.seed,config.dataset.val.size,config.dataset.train.size,config.version,config.ExchangeOfThought.UsePersonaPromts,config.api,config.ExchangeOfThought.baseagent
158,7p4bjvgt,,run-37885bb8,run-37885bb8,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T19:19:21Z,finished,BootstrapFewShot,Report,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
159,vr2t1f0b,,run-b73cab8c,run-b73cab8c,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T19:26:28Z,finished,BootstrapFewShot,Report,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
160,218fdoof,,run-ed5a3323,run-ed5a3323,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T19:37:39Z,finished,BootstrapFewShot,multi_4,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
161,2ykfvix4,,run-48177bfe,run-48177bfe,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T19:46:22Z,finished,BootstrapFewShot,multi_4,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
162,db9mkwjf,,run-5be9b84a,run-5be9b84a,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T19:59:45Z,finished,BootstrapFewShot,bigram,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
163,a5sd4n0v,,run-a2cd92d9,run-a2cd92d9,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T20:09:00Z,finished,BootstrapFewShot,bigram,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
164,9m7pbfo8,,run-4c179459,run-4c179459,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T20:26:32Z,finished,BootstrapFewShot,Debate,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
165,3p9ol2dl,,run-f8f977c6,run-f8f977c6,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T20:38:29Z,finished,BootstrapFewShot,Debate,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
166,7a126aee,,run-072e08ca,run-072e08ca,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T20:46:42Z,finished,BootstrapFewShot,single,...,"You are Peter, a high school student recognize...","You are Peter, a strategic synthesizer who int...","You are Lilly, a transformative catalyst who b...",39.0,80.0,20.0,v2,True,openai,NaN
168,7ebmb2b2,,run-9c843ee3,run-9c843ee3,63vzqpmb,911bda46c22e8e6c97b473460e490d2ce1822052,2024-12-16T20:48:40Z,finished,BootstrapFewShot,Report,...,None,None,None,39.0,80.0,20.0,v2,False,openai,NaN


In [203]:
df_copy["summaryMetrics._wandb.runtime"] = round(df_copy["summaryMetrics._wandb.runtime"] / 60, 2)
df_copy["config.ExchangeOfThought.rounds"] = df_copy["config.ExchangeOfThought.rounds"].astype(int)
df_copy["summaryMetrics.metric_vector_search_weave.map25_score.mean"] = df_copy["summaryMetrics.metric_vector_search_weave.map25_score.mean"] * 100

In [204]:
final_columns = {
    "config.ExchangeOfThought.mode" : "mode",
    "config.ExchangeOfThought.rounds" : "rounds",
    "config.ExchangeOfThought.UsePersonaPromts" : "persona",
    #"config.Dspy.telepropmter.type" : "telepropmter",
    #"config.A" : "uses_predefined_promts",
    "summaryMetrics.metric_vector_search_weave.map25_score.mean" : "mean_map25_score [%]",
    "summaryMetrics.usage_cost_cents" : "cost [in ct]",
    "summaryMetrics._wandb.runtime" : "runtime [min]", # rem
}

In [205]:
df_copy = df_copy[list(final_columns.keys())]

In [206]:
df_copy = df_copy.rename(columns=final_columns)

In [207]:
df_copy.sort_values(by=["mode", "persona"], ascending=False, inplace=True)

In [210]:
df_copy.reset_index(drop=True, inplace=True)

In [211]:
# NOTE - You need to copy the latex withouth the ' symbol


# Persona FALSE Table
df_copy_persona_false = df_copy[df_copy["persona"] == False]
df_copy_persona_false.drop("persona", axis=1, inplace=True)
latex_table = df_copy_persona_false.to_latex(index=True, escape=True, float_format="%.2f")
print(latex_table)

\begin{tabular}{llrrrr}
\toprule
 & mode & rounds & mean\_map25\_score [\%] & cost [in ct] & runtime [min] \\
\midrule
1 & single & 1 & 19.23 & 0.02 & 1.62 \\
4 & multi\_4 & 1 & 19.03 & 0.09 & 10.27 \\
5 & multi\_4 & 2 & 15.87 & 0.17 & 15.65 \\
8 & bigram & 1 & 19.24 & 0.11 & 6.80 \\
9 & bigram & 2 & 19.47 & 0.21 & 9.47 \\
12 & Report & 1 & 18.16 & 0.08 & 6.62 \\
13 & Report & 2 & 17.39 & 0.14 & 13.45 \\
16 & Debate & 1 & 19.86 & 0.10 & 7.53 \\
17 & Debate & 2 & 20.83 & 0.13 & 10.73 \\
\bottomrule
\end{tabular}



C:\Users\Amaan\AppData\Local\Temp\ipykernel_46672\906903385.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy_persona_false.drop("persona", axis=1, inplace=True)


In [212]:
# Persona True Table
df_copy_persona_true = df_copy[df_copy["persona"] == True]
df_copy_persona_true.drop("persona", axis=1, inplace=True)
latex_table = df_copy_persona_true.to_latex(index=True, escape=True, float_format="%.2f")
print(latex_table)

\begin{tabular}{llrrrr}
\toprule
 & mode & rounds & mean\_map25\_score [\%] & cost [in ct] & runtime [min] \\
\midrule
0 & single & 1 & 17.90 & 0.02 & 1.18 \\
2 & multi\_4 & 1 & 17.14 & 0.10 & 8.32 \\
3 & multi\_4 & 2 & 14.49 & 0.18 & 13.00 \\
6 & bigram & 1 & 19.26 & 0.12 & 8.87 \\
7 & bigram & 2 & 16.17 & 0.21 & 17.15 \\
10 & Report & 1 & 18.94 & 0.08 & 6.77 \\
11 & Report & 2 & 13.67 & 0.14 & 10.80 \\
14 & Debate & 1 & 20.75 & 0.10 & 11.58 \\
15 & Debate & 2 & 19.09 & 0.14 & 7.78 \\
\bottomrule
\end{tabular}



C:\Users\Amaan\AppData\Local\Temp\ipykernel_46672\3914812985.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy_persona_true.drop("persona", axis=1, inplace=True)
